In [1]:
import pandas as pd

import agrupa_excel_v2 as ag

import pandas as pd
import re
from unidecode import unidecode
import os

from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, PatternFill, Font, numbers
from openpyxl.worksheet.table import Table, TableStyleInfo

In [2]:
import os

def listar_arquivos(pasta):
    try:
        # Verifica se o caminho fornecido é uma pasta válida
        if not os.path.isdir(pasta):
            raise NotADirectoryError(f"O caminho {pasta} não é uma pasta válida.")

        # Lista todos os arquivos na pasta com caminho completo
        arquivos = []
        for item in os.listdir(pasta):
            caminho_completo = os.path.join(pasta, item)
            if os.path.isfile(caminho_completo):
                arquivos.append(caminho_completo)

        return arquivos

    except Exception as e:
        return str(e)

# Exemplo de uso
caminho_pasta = r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\Historico\Mensal'
arquivos = listar_arquivos(caminho_pasta)

In [3]:
def obter_nome_arquivo(caminho_completo):
    return os.path.basename(caminho_completo)
nomes_arquivos = [obter_nome_arquivo(arquivo) for arquivo in arquivos]

In [4]:
ag.agrupaEx(r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\Historico\Mensal' ,r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\Historico\Agrupado\historico_agrupado.xlsx')

In [5]:
all_corretores = pd.read_excel(r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\Grades\Grades_agrupadas.xlsx')[['Corretor', 'Inspetor de producao']]
all_production = pd.read_excel(r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\Historico\Agrupado\historico_agrupado.xlsx')

In [6]:
merged_df = all_production.merge(all_corretores[['Corretor', 'Inspetor de producao']],
                                on='Corretor',
                                how='left',
                                suffixes=('', '_new'))

In [7]:
merged_df['Inspetor de producao_new'].fillna('Comercial', inplace=True)
merged_df['Ramo Seguro'].fillna('Zerado', inplace=True)

In [8]:
l_unique = merged_df['Inspetor de producao_new'].unique().tolist()

In [12]:
def export_to_excel_with_format(df, file_path):

# Exporta o DataFrame para um arquivo Excel
    df.to_excel(file_path, index=True)

    # Carrega o arquivo Excel
    wb = load_workbook(file_path)
    ws = wb.active



    # Cria um estilo para o cabeçalho
    header_style = NamedStyle(name='header')
    header_style.fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_style.font = Font(color='FFFFFF', bold=True)  # Título em negrito



    # Aplica a formatação ao cabeçalho de todas as colunas
    for col_idx in range(1, len(df.columns) + 2):
        ws.cell(row=1, column=col_idx).style = header_style

    # Ajusta a largura das colunas automaticamente
    for col_idx, column in enumerate(ws.columns, start=1):
        max_length = 0
        column_letter = ws.cell(row=1, column=col_idx).column_letter
        for cell in column:
            try:
                if cell.value is not None:
                    cell_length = len(str(cell.value))
                    if cell_length > max_length:
                        max_length = cell_length
            except:
                pass
        adjusted_width = max_length + 2  # Adiciona um pequeno espaço extra
        ws.column_dimensions[column_letter].width = max(10, adjusted_width)  # Largura mínima de 10

    # Adiciona um filtro à planilha
    ws.auto_filter.ref = ws.dimensions  # Define o autofiltro para todas as células preenchidas

    # Salva o arquivo Excel com a formatação aplicada
    wb.save(file_path)

In [13]:
folder_path = r'C:\Users\Thomas\Downloads\Base_v2\Emissao\Dados_Emissao\HisoricoMatriz'

ag.delete_all_contents(folder_path)

for i in l_unique:
    n_df = merged_df[merged_df['Inspetor de producao'] == i]
    n_df.reset_index(drop=True,inplace=True)
    n_df.index = n_df.index + 1

    pivot_df = n_df.pivot_table(index='Corretor', columns='Ramo Seguro', aggfunc='size', fill_value=0)
    #pivot_df = pivot_df.drop(columns=['Zerado'])
    colunas_piv = ['Auto', 'Residencial sob medida', 'Equipamento', 'Equipamento agrícola', 'Empresarial', 'Frota']
    colunas_piv_existe = pivot_df.columns.tolist()
    sequencia_c = [elemento for elemento in colunas_piv if elemento in colunas_piv_existe] #deixa a lista na sequencia certa
    pivot_df = pivot_df[sequencia_c]


    nome = unidecode(i)
    nome = re.sub(r'[\\/]+', '', nome)
    output_file = os.path.join(folder_path, nome+'.xlsx')
    export_to_excel_with_format(pivot_df, output_file)

In [2]:
# String de exemplo
texto = "1025,10"

# Selecionando o terceiro caractere
terceiro_caractere = texto[-3]

# Exibindo o resultado
print(terceiro_caractere)  # Saída: 'a'

,
